### loading the libaries

In [1]:
import warnings
warnings.filterwarnings("ignore")  #To ignore warnings

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq #Webscrapping libraries

import pandas as pd #To create a dataframe

### scraping start with user input

In [2]:
print("Hello Researcher !!")
foi = input("Select your field: ")
foi = foi.replace(" ","+") #User Input selecting the field

myurl='https://pubmed.ncbi.nlm.nih.gov/?term=cancer&filter=simsearch2.ffrft&filter=years.2015-2021&size=200&page='
myurl = myurl.replace("cancer",foi)
size=200
#myurl is the url link of the PubMed search engine


#Pubmed can display 10,000 of the results only. Since we are using the page size as 200, it displays only the 50 pages
works = True
while works:
  works = False
  n=int(input("Enter the first n number of pages you want: "))
  if n > 50:
    print("Maximum page limit exceeded, Enter a number less than or equal to 50")
    works = True
l=list(range(n+1)) #Selecting the first n pages of the search results
list_string = map(str, l)
l=list(list_string)
url=[]
for i in range(len(l)):
    x=myurl+l[i]
    url.append(x)
url=url[1:] #1-n

author_lst,journal_cit,journal_lst,title,id_lst=[],[],[],[],[]
counter=0

for link in range(len(url)):
    counter+=1
    print("Scraping the page : " ,counter) #Tracking the progress

    uClient=uReq(url[link]) #allows us to read the page
    page_html=uClient.read() #reading the entire page
    page_soup=soup(page_html,"html.parser")

    containers = page_soup.findAll('div',{'class':'docsum-citation full-citation'}) #for extracting the entire citation
    cons=page_soup.findAll('div',{'class':'docsum-content'})#for extracting the content
    container = containers[0]
    con=cons[0]

    for container in containers:
        author_cont=container.findAll('span',{'class':'docsum-authors full-authors'})#extract the authors
        author=author_cont[0].text
        author_lst.append(author)
        journal_cont=container.findAll('span',{'class':"docsum-journal-citation full-journal-citation"})
        journal=journal_cont[0].text
        journal_cit.append(journal)
        journal_name=container.findAll('span', {'class':"docsum-journal-citation short-journal-citation"})#extracting the journal name
        jour=journal_name[0].text
        journal_lst.append(jour)
        pmid_cont=container.findAll('span',{'class':"citation-part"})
        pmid=pmid_cont[0].text#extracting the PMID
        id_lst.append(pmid)
    for con in cons:
      t=con.findAll('a',{'class':'docsum-title'})
      tit=t[0].text
      title.append(tit)


Hello Researcher !!
Select your field: cancer
Enter the first n number of pages you want: 2
Scraping the page :  1
Scraping the page :  2


In [3]:
id_no=[]
ids=[]
x = "Keywords"
c=0
counter=0
for i in id_lst:
    s=i.split(" ")
    id_no.append(s[1])
key= []
for j in range(len(id_no)):
  try:
    c+=1
    print('Page being scrapped' ,c,)
    url2 = "https://pubmed.ncbi.nlm.nih.gov/" + id_no[j] +"/"
    uClient=uReq(url2)
    html=uClient.read()
    ps=soup(html,"html.parser")
    id_containers=ps.findAll('div',{'id':'abstract','class':'abstract'})
    if id_containers==[]:
      key.append('None')
    else:
      for id_cont in id_containers:
        keyword_cont=id_cont.findAll('p')
        none = True
        for i in keyword_cont:
          if x in str((i)):
            keyword=i.text
            key.append(keyword)
            none = False
            break
        if none:
          key.append('None')
  except:
    key.append('None')

Page being scrapped 1
Page being scrapped 2
Page being scrapped 3
Page being scrapped 4
Page being scrapped 5
Page being scrapped 6
Page being scrapped 7
Page being scrapped 8
Page being scrapped 9
Page being scrapped 10
Page being scrapped 11
Page being scrapped 12
Page being scrapped 13
Page being scrapped 14
Page being scrapped 15
Page being scrapped 16
Page being scrapped 17
Page being scrapped 18
Page being scrapped 19
Page being scrapped 20
Page being scrapped 21
Page being scrapped 22
Page being scrapped 23
Page being scrapped 24
Page being scrapped 25
Page being scrapped 26
Page being scrapped 27
Page being scrapped 28
Page being scrapped 29
Page being scrapped 30
Page being scrapped 31
Page being scrapped 32
Page being scrapped 33
Page being scrapped 34
Page being scrapped 35
Page being scrapped 36
Page being scrapped 37
Page being scrapped 38
Page being scrapped 39
Page being scrapped 40
Page being scrapped 41
Page being scrapped 42
Page being scrapped 43
Page being scrapped 

Page being scrapped 347
Page being scrapped 348
Page being scrapped 349
Page being scrapped 350
Page being scrapped 351
Page being scrapped 352
Page being scrapped 353
Page being scrapped 354
Page being scrapped 355
Page being scrapped 356
Page being scrapped 357
Page being scrapped 358
Page being scrapped 359
Page being scrapped 360
Page being scrapped 361
Page being scrapped 362
Page being scrapped 363
Page being scrapped 364
Page being scrapped 365
Page being scrapped 366
Page being scrapped 367
Page being scrapped 368
Page being scrapped 369
Page being scrapped 370
Page being scrapped 371
Page being scrapped 372
Page being scrapped 373
Page being scrapped 374
Page being scrapped 375
Page being scrapped 376
Page being scrapped 377
Page being scrapped 378
Page being scrapped 379
Page being scrapped 380
Page being scrapped 381
Page being scrapped 382
Page being scrapped 383
Page being scrapped 384
Page being scrapped 385
Page being scrapped 386
Page being scrapped 387
Page being scrap

### Preprocessing of the keywords

In [4]:
kw=[]
for i in key:
    if i != "None":
      i=i.strip().split(':')[1].replace("\n","").replace("  ",'').replace(';',', ')
      kw.append(i)
    else:
      kw.append("None")

In [5]:
journal=[]
year=[]
for i in journal_lst:
    j=i.split('.')
    journal.append(j[0])
    year.append(j[1])

In [6]:
df=pd.DataFrame()
df['Authors']=author_lst
df['title']=title
df['Full_citation']=journal_cit
df['Journal']=journal
df['year']=year
df['PMID']=id_lst
df['Keywords']=kw

### save the scraped file in .csv format with user defined name

In [7]:
df['title']=df['title'].str.replace('\n','')
df['year'].value_counts()
dfname = input("enter the file name with .csv format:  ")
df.to_csv(dfname,index=False)

enter the file name with .csv format:  new_scraped.csv


In [8]:
import json
import pymongo
from pymongo import MongoClient
import os
import sys

### upload in mongodb

In [9]:
db = input("dbname:  ")
colc = input("collection name:   ")
path = input("input path:  ")
class MongoDB(object):
    def __init__(self, dBName=None, collectionName=None):
        self.dBName = dBName
        self.collectionName = collectionName
        self.client = MongoClient("localhost", 27017, maxPoolSize=50)
        self.DB = self.client[self.dBName]
        self.collection = self.DB[self.collectionName]
    def InsertData(self, path=None):
        df = pd.read_csv(path)
        data = df.to_dict('records')
        self.collection.insert_many(data, ordered=False)
        print("All the Data has been Exported to Mongo DB Server")
if __name__ == "__main__":
    mongodb = MongoDB(dBName = db, collectionName=colc)
    mongodb.InsertData(path=path)

dbname:  new-data
collection name:   cancer_new
input path:  new_scraped.csv
All the Data has been Exported to Mongo DB Server


### the below cells are data extract from mongodb

In [10]:
dbn = input("enter the db name to access:  ")
co = input("enter the collection name to find:  ")
client = pymongo.MongoClient("mongodb://localhost:27017/") 

db = client[dbn] 
col = db[co]
x = col.find()

for i in x:
    print(i)

enter the db name to access:  new-data
enter the collection name to find:  cancer_new
{'_id': ObjectId('5fe8bf5bf817814232c866d6'), 'Authors': 'Goral V.', 'title': '                Pancreatic Cancer: Pathogenesis and Diagnosis.              ', 'Full_citation': 'Asian Pac J Cancer Prev. 2015;16(14):5619-24. doi: 10.7314/apjcp.2015.16.14.5619.', 'Journal': 'Asian Pac J Cancer Prev', 'year': 2015, 'PMID': 'PMID: 26320426', 'Keywords': 'None'}
{'_id': ObjectId('5fe8bf5bf817814232c866d7'), 'Authors': 'Torre LA, Siegel RL, Ward EM, Jemal A.', 'title': '                Global Cancer Incidence and Mortality Rates and Trends--An Update.              ', 'Full_citation': 'Cancer Epidemiol Biomarkers Prev. 2016 Jan;25(1):16-27. doi: 10.1158/1055-9965.EPI-15-0578. Epub 2015 Dec 14.', 'Journal': 'Cancer Epidemiol Biomarkers Prev', 'year': 2016, 'PMID': 'PMID: 26667886', 'Keywords': 'None'}
{'_id': ObjectId('5fe8bf5bf817814232c866d8'), 'Authors': 'Torre LA, Islami F, Siegel RL, Ward EM, Jemal A.', 't

In [11]:
pd.DataFrame.from_dict(i, orient='index')

,0
_id,5fe8bf5bf817814232c86865
Authors,[No authors listed]
title,A Vulnerable Side to MYC-Drive...
Full_citation,Cancer Discov. 2015 Nov;5(11):1116. doi: 10.11...
Journal,Cancer Discov
year,2015
PMID,PMID: 26403714
Keywords,None


In [12]:
dbn = input("enter the db name to access:  ")
co = input("enter the collection name to find:  ")
year = int(input("enter the year:  "))
Journal = input(("enter the Journal:  "))
client = pymongo.MongoClient("mongodb://localhost:27017/") 
  
myquery = {'year':year, 'Journal':Journal}
# db = client["pubmed_webscrape"] 
db = client[dbn] 
col = db[co]
# x = col.find()  

# col = db["brain"] 
  
x = col.find(myquery) 

for i in x:
    print(i)

enter the db name to access:  new-data
enter the collection name to find:  cancer_new
enter the year:  2018
enter the Journal:  Cancer Discov
{'_id': ObjectId('5fe4c72cf69a490e6b1c538b'), 'Authors': 'Wang J, Yao Z, Jonsson P, Allen AN, Qin ACR, Uddin S, Dunkel IJ, Petriccione M, Manova K, Haque S, Rosenblum MK, Pisapia DJ, Rosen N, Taylor BS, Pratilas CA.', 'title': '                A Secondary Mutation in BRAF Confers Resistance to RAF Inhibition in a BRAF(V600E)-Mutant Brain Tumor.              ', 'Full_citation': 'Cancer Discov. 2018 Sep;8(9):1130-1141. doi: 10.1158/2159-8290.CD-17-1263. Epub 2018 Jun 7.', 'Journal': 'Cancer Discov', 'year': 2018, 'PMID': 'PMID: 29880583', 'Keywords': 'None'}
{'_id': ObjectId('5fe4c72cf69a490e6b1c64c4'), 'Authors': '[No authors listed]', 'title': '                With GBM, T Cells May Be Stuck in Bone Marrow.              ', 'Full_citation': 'Cancer Discov. 2018 Oct;8(10):1203-1204. doi: 10.1158/2159-8290.CD-NB2018-117. Epub 2018 Sep 5.', 'Journal':